In [62]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import List
from usz_neuro_conversion.common import (
    SessionContext,
    NixContext,
    get_metadata_row,
    read_nix,
    get_date,
)
import nixio
from pynwb import NWBFile

In [63]:
def _get_session_data(ctx: SessionContext) -> nixio.Block:
    return ctx.nix.blocks[f"Data_Subject_{ctx.subject:02}_Session_{ctx.session:02}"]


def create_context(subject: int, session: int) -> SessionContext:
    nix_context = NixContext(
        subject, session, project="Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM"
    )
    nix = read_nix(nix_context)
    general = nix.sections["General"]
    nwb = NWBFile(
        session_description="Running experiment as described in the the experiment description",
        identifier=f"Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM_subject{subject:02}_session{session:02}",
        lab=general.props["Recording location"].values[0],
        institution="Universitätsspital Zürich, 8091 Zurich, Switzerland",  # Broken UTF-8 in file
        experimenter="Boran, Ece",  # TODO is this right?
        session_start_time=get_date(nix_context),
        keywords=[
            "Neuroscience",
            "Electrophysiology",
            "Human",
            "Awake",
            "Local field potential",
            "Neuronal action potential",
            "Spikes",
            "Medial temporal lobe",
            "Hippocampus",
            "Entorhinal cortex",
            "Amygdala",
            "Scalp EEG",
            "Intracranial EEG",
            "Cognitive task",
            "Verbal working memory",
            "Epilepsy",
        ],
    )
    return nix_context.to_session_context(nix, nwb)

Read the data we already have from existing nix files. The mapping from verbal to visual participant comes from "../in/metadata/participant_mapping.csv"

In [72]:
verbal_to_visual = {
    6: 1,
    8: 3,
    4: 4,
    3: 9,
    7: 10,
    1: 12,
    2: 13,
}
visual_locations = pd.DataFrame(columns=["participant", "electrode", "X", "Y", "Z", "location"])
for verbal_i, visual_i in verbal_to_visual.items():
    ctx = create_context(verbal_i, 1)
    _general = ctx.nix.sections["General"]

    session = ctx.nix.blocks[f"Data_Subject_0{verbal_i}_Session_01"]
    _electrode_map_ieeg = (
        session
        .groups["iEEG electrode information"]
        .data_arrays["iEEG_Electrode_MNI_Coordinates"]
    )

    ieeg_electrode_locations = np.ndarray(_electrode_map_ieeg.shape)
    _electrode_map_ieeg.read_direct(ieeg_electrode_locations)

    ieeg_electrode_locations_df = pd.DataFrame(
        ieeg_electrode_locations, columns=["X", "Y", "Z"]
    )
    _labels = [
        label[1:] for label in
        session
        .groups["iEEG data"]
        .data_arrays["iEEG_Data_Trial_01"]
        .dimensions[0]
        .labels
    ]
    ieeg_electrode_locations_df.insert(3, "electrode", _labels)
    ieeg_electrode_locations_df.insert(0, "participant", visual_i)

    ieeg_electrode_information = (
        session
        .groups["iEEG electrode information"]
        .data_arrays
    )

    names_and_locations = []
    for source in ieeg_electrode_information["iEEG_Electrode_Map"].sources:
        electrode_name = str(source.sources[0].name)[1:]
        location = source.sources[1].name
        metadata = get_metadata_row(ctx.to_nix_context())
        electrodes_in_soz = metadata["Electrodes in seizure onset zone (SOZ)"].split(",")
        is_in_soz = electrode_name[:-1] in electrodes_in_soz
        names_and_locations.append([electrode_name, location, is_in_soz])

    names_and_locations = pd.DataFrame(names_and_locations, columns=["electrode", "location", "is_in_soz"])
    ieeg_electrode_locations_df = ieeg_electrode_locations_df.merge(names_and_locations, on="electrode")
    visual_locations = pd.concat([visual_locations, ieeg_electrode_locations_df])
print(visual_locations.to_string())

C:\Users\conta\AppData\Local\Temp\ipykernel_4044\2748175376.py:56: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  visual_locations = pd.concat([visual_locations, ieeg_electrode_locations_df])
C:\Users\conta\AppData\Local\Temp\ipykernel_4044\2748175376.py:56: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  visual_locations = pd.concat([visual_locations, ieeg_electrode_locations_df])
C:\Users\conta\AppData\Local\Temp\ipykernel_4044\2748175376.py:56: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  visual_locations = pd.concat([visual_locations, ieeg_electrode_locations_df])
C:\Users\conta\AppData\Local\Temp\ipyker

   participant electrode          X          Y          Z                                                                         location is_in_soz
0            1      AHL1 -24.985748 -11.907035 -17.444419                                Hipp, Left Hippocampus rHipp, rostral hippocampus      True
1            1      AHL2 -28.303377 -14.991270 -15.678862                                Hipp, Left Hippocampus rHipp, rostral hippocampus      True
2            1      AHL3 -31.949166 -17.267229 -13.824129                                 Hipp, Left Hippocampus cHipp, caudal hippocampus      True
3            1      AHL4 -36.525137 -20.475806 -11.981470                                 Hipp, Left Hippocampus cHipp, caudal hippocampus      True
4            1      AHL5 -40.884593 -23.053460 -10.330847                                                                   no_label_found      True
5            1      AHL6 -45.876287 -25.329025  -8.826048          MTG, Left Middle Temporal Gyrus aSTS, a

C:\Users\conta\AppData\Local\Temp\ipykernel_4044\2748175376.py:56: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  visual_locations = pd.concat([visual_locations, ieeg_electrode_locations_df])


Read the data we don't already have from the excel file

In [74]:
@dataclass(frozen=True)
class ExcelMetadata:
    visual_participant: int
    initials: str
    cols: str
    rows: str
    in_soz: List[str]


excel_metadata = [
    ExcelMetadata(
        visual_participant=2,
        initials="SP",
        cols="E:G,J:K",
        rows="2:33",
        in_soz=["PHR", "AHR"]
    ),
    ExcelMetadata(
        visual_participant=5,
        initials="PN",
        cols="A:E",
        rows="2:33",
        in_soz=["LL", "PL"]
    ),
    ExcelMetadata(
        visual_participant=6,
        initials="SA",
        cols="B:D,J:K",
        rows="2:65",
        in_soz=["AIR", "AHL"]
    ),
    ExcelMetadata(
        visual_participant=7,
        initials="NP_2018",
        cols="B:F",
        rows="2:25",
        in_soz=["PHL"]
    ),
    ExcelMetadata(
        visual_participant=8,
        initials="DG",
        cols="A:C,M,O",
        rows="2:65",
        in_soz=["AHL1", "AHL2"]
    ),
    ExcelMetadata(
        visual_participant=11,
        initials="WT",
        cols="A:E",
        rows="2:41",
        in_soz=["FR"]
    ),
]

In [78]:
path = "../in/metadata/electrode_locations_mr_ct.xlsx"
excel = pd.DataFrame()
for metadata in excel_metadata:
    skiprows = int(metadata.rows.split(":")[0]) - 1
    nrows = int(metadata.rows.split(":")[1]) - skiprows
    sheet = pd.read_excel(
        io=path,
        sheet_name=metadata.initials,
        header=None,
        names=["X", "Y", "Z", "electrode", "location"],
        usecols=metadata.cols,
        skiprows=skiprows,
        nrows=nrows,
    )
    sheet.insert(0, "participant", metadata.visual_participant)
    # Trim quotation marks from location and electrode names
    sheet["location"] = sheet["location"].str.strip("\"").str.strip("'")
    sheet["electrode"] = sheet["electrode"].str.strip("\"").str.strip("'")
    # remove initial 'm'
    sheet["electrode"] = sheet["electrode"].str[1:]
    is_in_soz = (metadata.in_soz[0][-1].isalnum() & sheet["electrode"].isin(metadata.in_soz)) | (
        sheet["electrode"].str[:-1].isin(metadata.in_soz))
    sheet.insert(6, "is_in_soz", is_in_soz)
    excel = pd.concat([excel, sheet])
print(excel.to_string())

    participant          X          Y          Z electrode                                                                                      location  is_in_soz
0             2  29.463288 -12.128326 -16.577644      AHR1                                            Hipp, Right Hippocampus rHipp, rostral hippocampus       True
1             2  32.714867 -12.743193 -15.902830      AHR2                                            Hipp, Right Hippocampus rHipp, rostral hippocampus       True
2             2  37.619680 -14.488964 -15.019943      AHR3                                                                                no_label_found       True
3             2  41.527917 -16.639051 -14.043265      AHR4                                                                                no_label_found       True
4             2  46.905129 -18.019011 -13.058225      AHR5                                                                                no_label_found       True
5             2 

In [80]:
visual_locations = pd.concat([visual_locations, excel])
visual_locations.sort_values(by=["participant", "electrode"], inplace=True)
visual_locations.reset_index(drop=True, inplace=True)
# replace location "no_label_found" with empty string
visual_locations["location"].replace("no_label_found", "", inplace=True)
print(visual_locations.to_string())

    participant electrode          X          Y          Z                                                                                      location is_in_soz
0             1      AHL1 -24.985748 -11.907035 -17.444419                                             Hipp, Left Hippocampus rHipp, rostral hippocampus      True
1             1      AHL2 -28.303377 -14.991270 -15.678862                                             Hipp, Left Hippocampus rHipp, rostral hippocampus      True
2             1      AHL3 -31.949166 -17.267229 -13.824129                                              Hipp, Left Hippocampus cHipp, caudal hippocampus      True
3             1      AHL4 -36.525137 -20.475806 -11.981470                                              Hipp, Left Hippocampus cHipp, caudal hippocampus      True
4             1      AHL5 -40.884593 -23.053460 -10.330847                                                                                                    True
5             1      A

In [81]:
visual_locations.to_csv("../out/metadata/visual_task_electrodes.csv", index=False)